In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from useful_functions import import_file, remove_before, remove_zeros, make_bins, make_equal_bins

In [2]:
import_file('Alta', '2P01')

FileNotFoundError: Sensor_type moet met een hoofdletter beginnen!.
Voor sensor_number, kijk naar hoe de file heet en gebruik daarvan dus de eerste 4 karakters'
Examples: 'Data_clean/Locus_sensors/B.253_processed.xlsx', 'Data_clean/Alta_sensors/B.254_processed.xlsx'